# Tax units with single adult and 4+ kids

[CBPP's 2018 Federal Jobs Guarantee proposal](https://www.cbpp.org/research/full-employment/the-federal-job-guarantee-a-policy-to-achieve-permanent-full-employment#_ftn1) sets a minimum wage of \$24,600 in order to meet the poverty line for a family of four. This notebook calculates the number of tax units where this would be insufficient to lift one out of poverty: those with a single adult and 4+ kids.

![img](https://imgur.com/5Km1yLO.png)

*Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis*

## Setup

### Imports

In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np

In [2]:
tc.__version__

'0.20.1'

### Settings

In [3]:
# Specify number of decimals in tables.
pd.set_option('precision', 2)

## Generate data

### Load data

Generate a set of normal CPS records for 2018 using `Calculator`, then extract the dataframe.

In [4]:
calc = tc.Calculator(records=tc.Records.cps_constructor(), 
                     policy=tc.Policy(), verbose=False)
calc.advance_to_year(2018)

In [5]:
df = calc.dataframe(['nu18', 'XTOT', 's006', 'e00200'])

### Preprocess

In [6]:
df['n18'] = df.XTOT - df.nu18
df['has_wages'] = df.e00200 > 0
df['nu18_m'] = df.nu18 * df.s006 / 1e6
df['nu18_4plus'] = df.nu18 > 3
df['adults'] = np.where(df.n18 > 1, '2+', df.n18.astype(int).astype(str))
df['nu18_4plus_01adult'] = (df.adults != '2+') & df.nu18_4plus

## Analysis

In [7]:
df.pivot_table(index='adults', columns='nu18_4plus', values='nu18_m',
               aggfunc=sum, margins=True)

nu18_4plus,False,True,All
adults,,,
0,2.47,4.82e-03,2.48
1,21.26,2.75e+00,24.01
2+,47.52,8.02e+00,55.53
All,71.25,1.08e+01,82.02


In [8]:
df.pivot_table(index='nu18_4plus_01adult', columns='has_wages', 
               values='nu18_m', aggfunc=sum, margins=True)

has_wages,False,True,All
nu18_4plus_01adult,,,
False,9.66,69.61,79.27
True,0.89,1.86,2.75
All,10.56,71.47,82.02


In [9]:
nu18_m_share_wages = (
    df[df.has_wages & df.nu18_4plus_01adult].nu18_m.sum() / 
    df[df.has_wages].nu18_m.sum())
nu18_m_share_no_wages = (
    df[~df.has_wages & df.nu18_4plus_01adult].nu18_m.sum() / 
    df[~df.has_wages].nu18_m.sum())
nu18_m_share = df[df.nu18_4plus_01adult].nu18_m.sum() / df.nu18_m.sum()

In [10]:
print('The share of children in tax units with 0-1 adults and 4+ children ' + 
      'is {:.1f}%'.format(nu18_m_share * 100) +
      ': {:.1f}% of tax units with wages'.format(nu18_m_share_wages * 100) +
      ' and {:.1f}% of tax units with no wages.'.format(
          nu18_m_share_no_wages * 100))

The share of children in tax units with 0-1 adults and 4+ children is 3.4%: 2.6% of tax units with wages and 8.5% of tax units with no wages.


In [11]:
df.pivot_table(index='nu18_4plus_01adult', columns='has_wages', 
               values='s006', aggfunc=sum, margins=True)

has_wages,False,True,All
nu18_4plus_01adult,,,
False,4.58e+07,1.24e+08,1.69e+08
True,2.01e+05,4.21e+05,6.21e+05
All,4.60e+07,1.24e+08,1.70e+08


In [12]:
s006_share_wages = (
    df[df.has_wages & df.nu18_4plus_01adult].s006.sum() / 
    df[df.has_wages].s006.sum())
s006_share_no_wages = (
    df[~df.has_wages & df.nu18_4plus_01adult].s006.sum() / 
    df[~df.has_wages].s006.sum())
s006_share = df[df.nu18_4plus_01adult].s006.sum() / df.s006.sum()

In [13]:
print('The share of tax units with 0-1 adults and 4+ children ' + 
      'is {:.1f}%'.format(s006_share * 100) +
      ': {:.1f}% of tax units with wages'.format(s006_share_wages * 100) +
      ' and {:.1f}% of tax units with no wages.'.format(
          s006_share_no_wages * 100))

The share of tax units with 0-1 adults and 4+ children is 0.4%: 0.3% of tax units with wages and 0.4% of tax units with no wages.
